In [2]:
from gensim.models.ldamodel import LdaModel
import pandas as pd

from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

from gensim.models import TfidfModel

from collections import Counter
import itertools

# Adidas

In [3]:
def createLDA(column_name, im_cap_tf, n_topics = 5, alpha_val = 0.5, eta_val=0.5, top_n = None):
    
    if top_n != None:
        l = []
        for i in im_cap_tf[column_name]:
            l+=i
    
        wc = Counter(l)
        wc.most_common()
        
        mc_adidas = wc.most_common()[top_n:]
        mc_adidas_list = [i[0] for i in mc_adidas]
        
        texts = im_cap_tf[column_name].apply(lambda x : list(set(x).intersection(set(mc_adidas_list))))
    
    else:
        texts = im_cap_tf[column_name]
        
    # Create a corpus from a list of texts
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # Train the model on the corpus.
    lda = LdaModel(corpus, num_topics=n_topics, alpha=alpha_val, eta = eta_val, id2word = dictionary)
    
    for p in lda.print_topics(num_topics=n_topics, num_words=10):
        print(p[1])
    
    return lda

def createLDA_TFIDF(column_name, im_cap_tf, n_topics = 5, alpha_val = 0.5, eta_val=0.5, top_n = None):
    
    if top_n != None:
        l = []
        for i in im_cap_tf[column_name]:
            l+=i
    
        wc = Counter(l)
        wc.most_common()
        
        mc_adidas = wc.most_common()[top_n:]
        mc_adidas_list = [i[0] for i in mc_adidas]
        
        texts = im_cap_tf[column_name].apply(lambda x : list(set(x).intersection(set(mc_adidas_list))))
    
    else:
        texts = im_cap_tf[column_name]
        
    # Create a corpus from a list of texts
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    model = TfidfModel(corpus)  # fit model

    tfidf_corpus = model[corpus]  # apply model to the first corpus document

    # Train the model on the corpus.
    lda = LdaModel(corpus, num_topics=n_topics, alpha=alpha_val,eta = eta_val, id2word = dictionary)
    
    for p in lda.print_topics(num_topics=n_topics, num_words=10):
        print(p[1])
    
    return lda

In [4]:
def get_word2topic(lda):
    lda_word_topic_df = pd.DataFrame(lda.get_topics()).T
    lda_word_topic_df.index = lda.id2word.values()
    return lda_word_topic_df

## Only Captions

In [13]:
data = pd.read_csv("./Adidas/adidas_caption_tf.csv", index_col=0)
print(data.columns)
column_name = 'caption_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['really', 'shawnjohnsons', 'toddantonyphoto', 'nosajthe', 'broken',
       'tired', 'collect', 'brasileira', 'best', 'happiness',
       ...
       'universal', 'narrow', 'bowerman', 'corner', 'rise', 'tomckean',
       'fencer', 'gomofarah', 'racer', 'caption_list'],
      dtype='object', length=2832)


In [8]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('adidas_topic_model_caption_tf.xlsx')
writer_tfidf = pd.ExcelWriter('adidas_topic_model_caption_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.017*"create" + 0.008*"heretocreate" + 0.007*"run" + 0.006*"adidasoriginal" + 0.006*"make" + 0.005*"time" + 0.005*"n't" + 0.004*"adida" + 0.004*"week" + 0.004*"``"
0.012*"see" + 0.007*"possibility" + 0.007*"heretocreate" + 0.006*"--" + 0.006*"bio" + 0.006*"link" + 0.005*"get" + 0.005*"make" + 0.005*"world" + 0.005*"adidasoriginal"
0.010*"heretocreate" + 0.010*"new" + 0.008*"make" + 0.007*"``" + 0.007*"see" + 0.006*"world" + 0.004*"available" + 0.004*"--" + 0.004*"sport" + 0.004*"run"

------------------------LDA with TFIDF ----------------------- 

0.012*"heretocreate" + 0.009*"new" + 0.007*"make" + 0.006*"--" + 0.006*"run" + 0.006*"create" + 0.005*"see" + 0.005*"year" + 0.005*"adida" + 0.005*"boost"
0.014*"create" + 0.007*"heretocreate" + 0.006*"make" + 0.006*"``" + 0.006*"new" + 0.006*"available" + 0.005*"see" + 0.005*"today" + 0.004*"day" + 0.004*"adida"
0.012*"see" + 0

0.003*"futurecraft" + 0.003*"head" + 0.003*"tap" + 0.003*"go" + 0.003*"find" + 0.002*"mean" + 0.002*"history" + 0.002*"ever" + 0.002*"introduce" + 0.002*"shop"
0.004*"story" + 0.003*"'s" + 0.003*"fast" + 0.003*"top" + 0.003*"store" + 0.003*"performance" + 0.003*"tap" + 0.003*"city" + 0.002*"ever" + 0.002*"next"
0.004*"drop" + 0.003*"creativity" + 0.003*"never" + 0.003*"inspire" + 0.003*"creator" + 0.003*"store" + 0.003*"collection" + 0.003*"hometeam" + 0.003*"speedtakes" + 0.003*"history"
# Topics 3 Alpha 0.5 ETA val 0.7 top_n None

------------------------LDA without TFIDF ----------------------- 

0.012*"heretocreate" + 0.009*"create" + 0.006*"--" + 0.006*"make" + 0.005*"see" + 0.005*"adidasoriginal" + 0.005*"available" + 0.005*"time" + 0.005*"new" + 0.004*"world"
0.009*"see" + 0.006*"possibility" + 0.005*"make" + 0.005*"create" + 0.005*"new" + 0.004*"get" + 0.004*"world" + 0.004*"adidasoriginal" + 0.004*"shoe" + 0.003*"together"
0.006*"heretocreate" + 0.005*"create" + 0.005*"bio" + 

0.004*"never" + 0.004*"creator" + 0.003*"'s" + 0.003*"tap" + 0.003*"inspire" + 0.003*"go" + 0.003*"speedtakes" + 0.003*"ever" + 0.003*"collection" + 0.003*"turn"
0.003*"story" + 0.003*"head" + 0.003*"collection" + 0.003*"pack" + 0.003*"futurecraft" + 0.003*"worldcup" + 0.003*"play" + 0.003*"creator" + 0.003*"history" + 0.003*"find"
0.004*"store" + 0.004*"go" + 0.003*"drop" + 0.003*"hometeam" + 0.003*"together" + 0.003*"street" + 0.003*"worldwide" + 0.003*"start" + 0.003*"ever" + 0.003*"online"

------------------------LDA with TFIDF ----------------------- 

0.004*"speedtakes" + 0.003*"never" + 0.003*"head" + 0.003*"drop" + 0.003*"go" + 0.003*"street" + 0.003*"worldwide" + 0.003*"collection" + 0.002*"ever" + 0.002*"life"
0.005*"store" + 0.005*"'s" + 0.004*"online" + 0.003*"history" + 0.003*"creator" + 0.003*"story" + 0.003*"drop" + 0.003*"turn" + 0.003*"performance" + 0.003*"together"
0.003*"ever" + 0.003*"join" + 0.003*"play" + 0.003*"tap" + 0.003*"hometeam" + 0.003*"story" + 0.003*"s

0.004*"week" + 0.003*"first" + 0.003*"today" + 0.003*"year" + 0.003*"head" + 0.003*"store" + 0.003*"story" + 0.002*"'s" + 0.002*"kickoftheweek" + 0.002*"use"
0.004*"n't" + 0.003*"creator" + 0.003*"year" + 0.003*"inspire" + 0.003*"go" + 0.002*"share" + 0.002*"week" + 0.002*"game" + 0.002*"together" + 0.002*"tap"
0.003*"day" + 0.003*"game" + 0.003*"never" + 0.003*"today" + 0.003*"boost" + 0.003*"collection" + 0.003*"performance" + 0.003*"play" + 0.003*"store" + 0.002*"every"
# Topics 3 Alpha 0.7 ETA val 0.7 top_n 30

------------------------LDA without TFIDF ----------------------- 

0.003*"play" + 0.003*"collection" + 0.003*"store" + 0.003*"'s" + 0.003*"celebrate" + 0.003*"inspire" + 0.002*"tap" + 0.002*"history" + 0.002*"go" + 0.002*"never"
0.003*"ever" + 0.003*"start" + 0.003*"drop" + 0.003*"creator" + 0.003*"'s" + 0.003*"story" + 0.002*"turn" + 0.002*"join" + 0.002*"hometeam" + 0.002*"help"
0.003*"creator" + 0.003*"speedtakes" + 0.003*"futurecraft" + 0.002*"store" + 0.002*"together" 

# Topics 5 Alpha 0.5 ETA val 0.5 top_n None

------------------------LDA without TFIDF ----------------------- 

0.012*"heretocreate" + 0.007*"new" + 0.006*"create" + 0.006*"world" + 0.006*"see" + 0.005*"make" + 0.005*"--" + 0.004*"``" + 0.004*"store" + 0.003*"adida"
0.006*"create" + 0.005*"see" + 0.004*"make" + 0.004*"available" + 0.004*"adidasoriginal" + 0.003*"world" + 0.003*"day" + 0.003*"new" + 0.003*"get" + 0.003*"every"
0.007*"heretocreate" + 0.006*"run" + 0.006*"create" + 0.006*"make" + 0.005*"--" + 0.004*"future" + 0.004*"sport" + 0.003*"world" + 0.003*"adidasoriginal" + 0.003*"creator"
0.007*"create" + 0.006*"heretocreate" + 0.006*"make" + 0.005*"link" + 0.005*"``" + 0.005*"bio" + 0.005*"see" + 0.004*"new" + 0.004*"world" + 0.004*"get"
0.009*"see" + 0.007*"possibility" + 0.005*"new" + 0.005*"create" + 0.004*"run" + 0.004*"heretocreate" + 0.004*"time" + 0.003*"make" + 0.003*"sport" + 0.003*"game"

------------------------LDA with TFIDF ----------------------- 

0.010*"see" + 0

...


------------------------LDA with TFIDF ----------------------- 

0.008*"heretocreate" + 0.007*"create" + 0.006*"see" + 0.006*"make" + 0.005*"get" + 0.005*"new" + 0.004*"bio" + 0.004*"time" + 0.004*"link" + 0.004*"possibility"
0.010*"create" + 0.009*"see" + 0.009*"heretocreate" + 0.008*"new" + 0.006*"make" + 0.005*"world" + 0.005*"possibility" + 0.004*"run" + 0.004*"ever" + 0.004*"available"
0.012*"see" + 0.008*"heretocreate" + 0.007*"available" + 0.007*"create" + 0.006*"new" + 0.006*"make" + 0.005*"world" + 0.005*"bio" + 0.005*"``" + 0.005*"adidasoriginal"
0.011*"heretocreate" + 0.011*"world" + 0.009*"create" + 0.006*"adidasoriginal" + 0.006*"week" + 0.005*"adida" + 0.005*"make" + 0.005*"see" + 0.005*"available" + 0.004*"run"
0.009*"heretocreate" + 0.008*"make" + 0.008*"create" + 0.008*"see" + 0.006*"run" + 0.005*"new" + 0.005*"time" + 0.005*"world" + 0.004*"year" + 0.004*"--"
0.009*"possibility" + 0.009*"heretocreate" + 0.008*"see" + 0.008*"make" + 0.008*"create" + 0.006*"new" + 0.00

# Topics 10 Alpha 0.7 ETA val None top_n 30

------------------------LDA without TFIDF ----------------------- 

0.004*"together" + 0.003*"history" + 0.003*"inspire" + 0.003*"ever" + 0.003*"futurecraft" + 0.003*"store" + 0.003*"performance" + 0.003*"watch" + 0.003*"collection" + 0.003*"creator"
0.003*"story" + 0.003*"creator" + 0.003*"futurecraft" + 0.003*"drop" + 0.003*"together" + 0.003*"collection" + 0.003*"win" + 0.003*"creativity" + 0.002*"worldwide" + 0.002*"never"
0.004*"store" + 0.004*"ever" + 0.003*"creator" + 0.003*"play" + 0.003*"next" + 0.003*"'s" + 0.003*"creativity" + 0.003*"drop" + 0.003*"never" + 0.003*"story"
0.004*"never" + 0.003*"creator" + 0.003*"tap" + 0.003*"go" + 0.003*"play" + 0.003*"store" + 0.003*"wave" + 0.003*"adidasfootball" + 0.003*"city" + 0.003*"performance"
0.004*"story" + 0.003*"ever" + 0.003*"never" + 0.003*"creator" + 0.003*"tap" + 0.002*"store" + 0.002*"shoe" + 0.002*"champion" + 0.002*"drop" + 0.002*"collection"
0.003*"ultraboost" + 0.003*"find" + 


------------------------LDA with TFIDF ----------------------- 

0.003*"bio" + 0.003*"link" + 0.002*"available" + 0.002*"--" + 0.002*"year" + 0.002*"time" + 0.002*"get" + 0.002*"change" + 0.002*"future" + 0.002*"ever"
0.003*"available" + 0.002*"link" + 0.002*"--" + 0.002*"adida" + 0.002*"future" + 0.002*"week" + 0.002*"store" + 0.002*"creator" + 0.002*"bio" + 0.002*"get"
0.003*"link" + 0.003*"--" + 0.003*"bio" + 0.002*"adida" + 0.002*"day" + 0.002*"available" + 0.002*"change" + 0.002*"speedtakes" + 0.002*"every" + 0.002*"week"
0.003*"bio" + 0.003*"time" + 0.002*"year" + 0.002*"adida" + 0.002*"link" + 0.002*"--" + 0.002*"game" + 0.002*"go" + 0.002*"get" + 0.002*"creator"
0.003*"--" + 0.003*"future" + 0.003*"change" + 0.002*"week" + 0.002*"store" + 0.002*"game" + 0.002*"year" + 0.002*"sport" + 0.002*"link" + 0.002*"today"
0.002*"--" + 0.002*"adida" + 0.002*"first" + 0.002*"time" + 0.002*"link" + 0.002*"'s" + 0.002*"sport" + 0.002*"story" + 0.002*"game" + 0.002*"available"
0.002*"availab

# Topics 10 Alpha 0.7 ETA val 0.7 top_n None

------------------------LDA without TFIDF ----------------------- 

0.004*"make" + 0.004*"see" + 0.004*"create" + 0.004*"heretocreate" + 0.003*"world" + 0.003*"new" + 0.003*"year" + 0.002*"--" + 0.002*"time" + 0.002*"sport"
0.004*"create" + 0.003*"make" + 0.003*"new" + 0.003*"heretocreate" + 0.003*"adidasoriginal" + 0.003*"--" + 0.002*"available" + 0.002*"run" + 0.002*"see" + 0.002*"creator"
0.005*"create" + 0.005*"heretocreate" + 0.003*"make" + 0.003*"new" + 0.003*"see" + 0.002*"world" + 0.002*"link" + 0.002*"every" + 0.002*"--" + 0.002*"get"
0.004*"see" + 0.004*"possibility" + 0.004*"heretocreate" + 0.004*"create" + 0.002*"new" + 0.002*"adidasoriginal" + 0.002*"make" + 0.002*"available" + 0.002*"time" + 0.002*"world"
0.004*"create" + 0.004*"make" + 0.004*"heretocreate" + 0.003*"see" + 0.003*"possibility" + 0.002*"new" + 0.002*"every" + 0.002*"``" + 0.002*"change" + 0.002*"world"
0.004*"heretocreate" + 0.004*"``" + 0.003*"create" + 0.003*"


------------------------LDA with TFIDF ----------------------- 

0.002*"year" + 0.002*"n't" + 0.002*"plastic" + 0.002*"ever" + 0.002*"start" + 0.002*"every" + 0.002*"shoe" + 0.002*"first" + 0.001*"'s" + 0.001*"help"
0.002*"n't" + 0.002*"'s" + 0.002*"never" + 0.002*"store" + 0.001*"today" + 0.001*"game" + 0.001*"drop" + 0.001*"first" + 0.001*"go" + 0.001*"week"
0.002*"week" + 0.002*"year" + 0.002*"boost" + 0.002*"game" + 0.002*"speedtakes" + 0.002*"ever" + 0.001*"go" + 0.001*"drop" + 0.001*"every" + 0.001*"store"
0.002*"week" + 0.002*"hometeam" + 0.002*"day" + 0.002*"tap" + 0.001*"today" + 0.001*"head" + 0.001*"year" + 0.001*"collection" + 0.001*"game" + 0.001*"every"
0.002*"week" + 0.002*"never" + 0.002*"creativity" + 0.001*"store" + 0.001*"creator" + 0.001*"story" + 0.001*"head" + 0.001*"year" + 0.001*"pack" + 0.001*"game"
0.002*"n't" + 0.002*"first" + 0.001*"game" + 0.001*"street" + 0.001*"go" + 0.001*"store" + 0.001*"today" + 0.001*"never" + 0.001*"year" + 0.001*"creator"
0.002*"n'

## Only Image Labels

In [9]:
data = pd.read_csv("./Adidas/adidas_label_tf.csv", index_col=0)
print(data.columns)
column_name = 'labels_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['farmworker', 'photograph', 'symbol', 'portrait', 'eyewear', 'toy',
       'flag', 'kickflip', 'concrete', 'bicycles',
       ...
       'leg', 'tire', 'music', 'sportswear', 'serveware', 'bookcase',
       'astronomical', 'freeze', 'labels', 'labels_list'],
      dtype='object', length=670)


In [10]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('adidas_topic_model_image_tf.xlsx')
writer_tfidf = pd.ExcelWriter('adidas_topic_model_image_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.029*"sports" + 0.028*"shoe" + 0.020*"equipment" + 0.020*"sportswear" + 0.019*"font" + 0.014*"blue" + 0.014*"uniform" + 0.013*"leg" + 0.013*"sport" + 0.013*"thigh"
0.036*"shoe" + 0.020*"sports" + 0.019*"leg" + 0.018*"photography" + 0.018*"fashion" + 0.018*"sky" + 0.017*"equipment" + 0.017*"sportswear" + 0.016*"footwear" + 0.013*"flash"
0.024*"shoe" + 0.023*"sleeve" + 0.021*"photography" + 0.020*"flash" + 0.020*"fashion" + 0.016*"-" + 0.016*"thigh" + 0.015*"human" + 0.014*"equipment" + 0.014*"sky"

------------------------LDA with TFIDF ----------------------- 

0.028*"shoe" + 0.022*"fashion" + 0.022*"sports" + 0.020*"sportswear" + 0.018*"equipment" + 0.016*"-" + 0.015*"thigh" + 0.015*"footwear" + 0.014*"flash" + 0.014*"font"
0.029*"photography" + 0.024*"flash" + 0.020*"equipment" + 0.019*"sky" + 0.018*"gesture" + 0.016*"sleeve" + 0.016*"font" + 0.016*"sports" + 0.014*"neck

0.017*"white" + 0.013*"shorts" + 0.012*"black" + 0.012*"muscle" + 0.011*"event" + 0.011*"sneakers" + 0.010*"and" + 0.010*"chin" + 0.010*"landscape" + 0.009*"plant"
0.016*"event" + 0.016*"entertainment" + 0.013*"and" + 0.013*"jaw" + 0.012*"black" + 0.011*"recreation" + 0.010*"short" + 0.010*"beard" + 0.010*"eyebrow" + 0.010*"arm"
0.019*"nature" + 0.019*"in" + 0.016*"design" + 0.014*"plant" + 0.014*"t" + 0.013*"electric" + 0.013*"leisure" + 0.011*"shorts" + 0.011*"arm" + 0.010*"automotive"
# Topics 3 Alpha 0.5 ETA val 0.7 top_n None

------------------------LDA without TFIDF ----------------------- 

0.032*"sports" + 0.030*"shoe" + 0.021*"equipment" + 0.019*"sportswear" + 0.018*"fashion" + 0.017*"sleeve" + 0.017*"sport" + 0.013*"flash" + 0.013*"uniform" + 0.013*"jersey"
0.033*"shoe" + 0.019*"leg" + 0.019*"equipment" + 0.018*"photography" + 0.018*"human" + 0.017*"thigh" + 0.014*"flash" + 0.013*"sportswear" + 0.013*"body" + 0.013*"and"
0.024*"font" + 0.019*"shoe" + 0.018*"sky" + 0.017*"fas

0.014*"nature" + 0.013*"smile" + 0.012*"event" + 0.011*"electric" + 0.011*"shorts" + 0.011*"recreation" + 0.011*"design" + 0.011*"entertainment" + 0.011*"eyebrow" + 0.011*"t"
0.014*"joint" + 0.013*"design" + 0.013*"shorts" + 0.012*"t" + 0.012*"white" + 0.011*"and" + 0.011*"nature" + 0.010*"electric" + 0.010*"shoulder" + 0.010*"in"
0.017*"plant" + 0.013*"black" + 0.012*"arm" + 0.012*"and" + 0.012*"event" + 0.011*"leisure" + 0.011*"tree" + 0.010*"in" + 0.010*"shorts" + 0.010*"grey"

------------------------LDA with TFIDF ----------------------- 

0.015*"electric" + 0.015*"and" + 0.015*"t" + 0.013*"nature" + 0.013*"in" + 0.013*"plant" + 0.010*"eyebrow" + 0.010*"leisure" + 0.010*"white" + 0.009*"event"
0.016*"shorts" + 0.015*"short" + 0.015*"arm" + 0.013*"sneakers" + 0.013*"entertainment" + 0.013*"muscle" + 0.012*"black" + 0.012*"design" + 0.012*"leisure" + 0.012*"event"
0.015*"nature" + 0.012*"plant" + 0.012*"shorts" + 0.011*"shoulder" + 0.011*"event" + 0.010*"in" + 0.010*"cloud" + 0.010*

0.013*"uniform" + 0.012*"short" + 0.012*"shirt" + 0.011*"event" + 0.011*"player" + 0.010*"blue" + 0.010*"leisure" + 0.010*"people" + 0.009*"t" + 0.009*"electric"
0.018*"knee" + 0.013*"and" + 0.012*"shorts" + 0.011*"white" + 0.011*"player" + 0.010*"shirt" + 0.010*"uniform" + 0.010*"blue" + 0.010*"neck" + 0.009*"t"
0.015*"blue" + 0.014*"body" + 0.013*"uniform" + 0.012*"nature" + 0.012*"outdoor" + 0.012*"neck" + 0.011*"black" + 0.011*"player" + 0.010*"arm" + 0.010*"in"
# Topics 3 Alpha 0.7 ETA val 0.7 top_n 30

------------------------LDA without TFIDF ----------------------- 

0.013*"eyebrow" + 0.012*"white" + 0.011*"event" + 0.010*"hair" + 0.010*"forehead" + 0.010*"short" + 0.010*"in" + 0.010*"and" + 0.010*"water" + 0.010*"shorts"
0.013*"t" + 0.013*"athletic" + 0.011*"electric" + 0.009*"black" + 0.009*"walking" + 0.009*"grey" + 0.009*"cloud" + 0.009*"white" + 0.008*"short" + 0.008*"and"
0.014*"shorts" + 0.014*"nature" + 0.012*"plant" + 0.012*"shoulder" + 0.011*"design" + 0.011*"joint" +

0.022*"sky" + 0.019*"fashion" + 0.019*"happy" + 0.017*"shoe" + 0.015*"sleeve" + 0.015*"font" + 0.014*"sportswear" + 0.013*"leg" + 0.013*"sports" + 0.012*"art"
0.028*"shoe" + 0.019*"photography" + 0.018*"flash" + 0.017*"human" + 0.017*"fashion" + 0.014*"sports" + 0.013*"-" + 0.013*"leg" + 0.012*"street" + 0.012*"sportswear"
0.018*"sleeve" + 0.017*"-" + 0.016*"thigh" + 0.015*"sport" + 0.015*"leg" + 0.014*"font" + 0.014*"flash" + 0.014*"fashion" + 0.014*"photography" + 0.013*"gesture"
0.044*"sports" + 0.036*"equipment" + 0.021*"jersey" + 0.017*"sport" + 0.017*"player" + 0.015*"bicycle" + 0.014*"football" + 0.014*"shoe" + 0.013*"sky" + 0.012*"uniform"
0.057*"shoe" + 0.020*"footwear" + 0.019*"fashion" + 0.018*"font" + 0.018*"blue" + 0.016*"photography" + 0.016*"sportswear" + 0.015*"thigh" + 0.015*"art" + 0.015*"flash"

------------------------LDA with TFIDF ----------------------- 

0.024*"sports" + 0.018*"fashion" + 0.017*"flash" + 0.015*"photography" + 0.015*"equipment" + 0.014*"shoe" + 0

...

0.025*"equipment" + 0.021*"shoe" + 0.015*"sports" + 0.014*"font" + 0.014*"photography" + 0.013*"flash" + 0.013*"sportswear" + 0.013*"sky" + 0.013*"sport" + 0.012*"human"
0.030*"sports" + 0.030*"shoe" + 0.022*"equipment" + 0.021*"leg" + 0.017*"sportswear" + 0.016*"font" + 0.016*"uniform" + 0.015*"footwear" + 0.015*"thigh" + 0.014*"sleeve"
0.034*"shoe" + 0.018*"photography" + 0.017*"sportswear" + 0.017*"thigh" + 0.016*"leg" + 0.016*"-" + 0.015*"sleeve" + 0.015*"sky" + 0.015*"font" + 0.014*"fashion"
0.044*"shoe" + 0.020*"sports" + 0.016*"photography" + 0.016*"equipment" + 0.016*"fashion" + 0.015*"footwear" + 0.015*"flash" + 0.014*"sportswear" + 0.013*"sleeve" + 0.013*"happy"
0.028*"shoe" + 0.022*"fashion" + 0.019*"font" + 0.018*"sports" + 0.017*"equipment" + 0.016*"photography" + 0.015*"flash" + 0.014*"gesture" + 0.013*"sportswear" + 0.012*"sport"
0.020*"photography" + 0.019*"sleeve" + 0.018*"shoe" + 0.018*"sports" + 0.018*"leg" + 0.016*"sportswear" + 0.016*"flash" + 0.016*"art" + 0.015*"

0.014*"t" + 0.014*"leisure" + 0.013*"black" + 0.012*"in" + 0.012*"white" + 0.011*"arm" + 0.011*"event" + 0.010*"shorts" + 0.010*"muscle" + 0.010*"short"
0.012*"arm" + 0.012*"event" + 0.012*"shorts" + 0.011*"electric" + 0.011*"eyebrow" + 0.010*"entertainment" + 0.010*"white" + 0.010*"and" + 0.010*"black" + 0.010*"short"
0.015*"electric" + 0.013*"plant" + 0.012*"arm" + 0.012*"shorts" + 0.012*"nature" + 0.011*"design" + 0.010*"hair" + 0.010*"white" + 0.010*"grass" + 0.010*"t"
0.014*"design" + 0.012*"shoulder" + 0.012*"shorts" + 0.012*"nature" + 0.011*"forehead" + 0.011*"plant" + 0.011*"eyebrow" + 0.010*"smile" + 0.010*"waist" + 0.009*"joint"
0.013*"plant" + 0.012*"automotive" + 0.012*"sneakers" + 0.011*"forehead" + 0.011*"white" + 0.011*"shorts" + 0.010*"entertainment" + 0.010*"eyebrow" + 0.010*"joint" + 0.010*"recreation"
0.014*"in" + 0.013*"event" + 0.013*"nature" + 0.011*"shorts" + 0.011*"entertainment" + 0.011*"black" + 0.011*"white" + 0.011*"recreation" + 0.010*"electric" + 0.010*"pl

0.015*"leg" + 0.015*"human" + 0.012*"footwear" + 0.010*"thigh" + 0.010*"-" + 0.010*"blue" + 0.010*"knee" + 0.009*"people" + 0.009*"street" + 0.009*"nature"
0.013*"gesture" + 0.013*"footwear" + 0.011*"leg" + 0.010*"blue" + 0.009*"human" + 0.008*"white" + 0.008*"shirt" + 0.008*"shorts" + 0.008*"thigh" + 0.008*"street"
0.014*"thigh" + 0.013*"leg" + 0.013*"street" + 0.011*"gesture" + 0.009*"-" + 0.009*"design" + 0.008*"player" + 0.008*"uniform" + 0.008*"blue" + 0.008*"art"
0.012*"-" + 0.012*"gesture" + 0.011*"sport" + 0.011*"neck" + 0.011*"blue" + 0.011*"happy" + 0.010*"street" + 0.010*"uniform" + 0.010*"human" + 0.009*"leg"
0.012*"footwear" + 0.012*"knee" + 0.011*"leg" + 0.011*"thigh" + 0.011*"human" + 0.010*"happy" + 0.010*"gesture" + 0.009*"art" + 0.008*"-" + 0.008*"joint"
0.014*"leg" + 0.013*"thigh" + 0.011*"-" + 0.011*"human" + 0.010*"art" + 0.010*"footwear" + 0.010*"happy" + 0.009*"gesture" + 0.009*"uniform" + 0.008*"body"
0.015*"thigh" + 0.013*"street" + 0.013*"human" + 0.012*"leg" 

0.018*"shoe" + 0.015*"flash" + 0.015*"fashion" + 0.014*"sleeve" + 0.014*"photography" + 0.012*"-" + 0.012*"happy" + 0.010*"sky" + 0.010*"equipment" + 0.010*"street"
0.018*"shoe" + 0.016*"sports" + 0.014*"font" + 0.014*"equipment" + 0.012*"sportswear" + 0.010*"fashion" + 0.010*"human" + 0.010*"sleeve" + 0.010*"art" + 0.010*"-"
0.020*"shoe" + 0.016*"photography" + 0.015*"sports" + 0.014*"flash" + 0.011*"leg" + 0.010*"art" + 0.010*"footwear" + 0.010*"gesture" + 0.010*"equipment" + 0.009*"sportswear"
0.021*"shoe" + 0.016*"sports" + 0.016*"fashion" + 0.013*"equipment" + 0.011*"footwear" + 0.010*"street" + 0.010*"thigh" + 0.010*"font" + 0.010*"sportswear" + 0.009*"sky"
0.043*"shoe" + 0.017*"sportswear" + 0.015*"sports" + 0.013*"font" + 0.012*"human" + 0.012*"photography" + 0.012*"outdoor" + 0.011*"equipment" + 0.011*"sport" + 0.011*"leg"
0.022*"shoe" + 0.012*"fashion" + 0.011*"sportswear" + 0.011*"leg" + 0.010*"sky" + 0.010*"sports" + 0.009*"equipment" + 0.009*"footwear" + 0.009*"font" + 0.0

0.011*"shirt" + 0.010*"player" + 0.010*"knee" + 0.009*"event" + 0.009*"plant" + 0.008*"blue" + 0.008*"shorts" + 0.008*"uniform" + 0.007*"neck" + 0.007*"nature"
0.009*"knee" + 0.008*"shorts" + 0.008*"uniform" + 0.008*"outdoor" + 0.008*"player" + 0.008*"smile" + 0.007*"body" + 0.007*"blue" + 0.007*"in" + 0.007*"nature"
0.009*"outdoor" + 0.008*"blue" + 0.008*"people" + 0.008*"knee" + 0.008*"body" + 0.007*"short" + 0.007*"white" + 0.007*"neck" + 0.007*"event" + 0.007*"shirt"
0.010*"uniform" + 0.010*"knee" + 0.009*"people" + 0.009*"nature" + 0.008*"in" + 0.008*"blue" + 0.008*"shirt" + 0.007*"player" + 0.007*"electric" + 0.007*"plant"
0.013*"outdoor" + 0.008*"blue" + 0.007*"t" + 0.007*"white" + 0.007*"body" + 0.007*"uniform" + 0.007*"shorts" + 0.007*"knee" + 0.007*"black" + 0.006*"and"
0.011*"uniform" + 0.010*"shirt" + 0.009*"blue" + 0.008*"design" + 0.008*"player" + 0.008*"t" + 0.008*"electric" + 0.008*"knee" + 0.007*"leisure" + 0.007*"shorts"
0.013*"uniform" + 0.009*"knee" + 0.009*"blue" +

## Image Labels + Caption

In [11]:
data = pd.read_csv("./Adidas/adidas_img_cap_tf.csv", index_col=0)
print(data.columns)
column_name = 'img_cap_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['fearless', 'strongerforit', 'coding', 'jaque1212', 'concrete',
       'leomessi', 'coreyrichproduction', 'feel', 'baller', 'unscripte',
       ...
       'writer', 'historical', 'distance', 'without', 'ritaora', 'bookcase',
       'eiger', 'argentinas', 'collide', 'img_cap_list'],
      dtype='object', length=2925)


In [12]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('adidas_topic_model_imgcap_tf.xlsx')
writer_tfidf = pd.ExcelWriter('adidas_topic_model_imgcap_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.013*"shoe" + 0.009*"sky" + 0.008*"photography" + 0.008*"sports" + 0.007*"equipment" + 0.007*"sleeve" + 0.007*"street" + 0.006*"gesture" + 0.006*"art" + 0.006*"flash"
0.013*"shoe" + 0.012*"sports" + 0.010*"equipment" + 0.009*"sport" + 0.008*"flash" + 0.007*"photography" + 0.007*"font" + 0.006*"leg" + 0.006*"sportswear" + 0.006*"human"
0.017*"shoe" + 0.011*"fashion" + 0.008*"sportswear" + 0.007*"thigh" + 0.006*"footwear" + 0.006*"font" + 0.006*"sport" + 0.006*"street" + 0.006*"-" + 0.006*"leg"

------------------------LDA with TFIDF ----------------------- 

0.009*"fashion" + 0.008*"-" + 0.006*"sleeve" + 0.006*"``" + 0.006*"create" + 0.005*"make" + 0.005*"black" + 0.005*"sky" + 0.005*"sports" + 0.004*"blue"
0.010*"photography" + 0.010*"shoe" + 0.008*"flash" + 0.008*"sky" + 0.008*"equipment" + 0.007*"sports" + 0.007*"street" + 0.006*"sport" + 0.006*"font" + 0.006*"fashion"
0

0.005*"plant" + 0.005*"--" + 0.004*"outdoor" + 0.004*"black" + 0.004*"nature" + 0.004*"head" + 0.004*"link" + 0.004*"shirt" + 0.004*"bio" + 0.004*"and"
0.007*"shorts" + 0.005*"electric" + 0.005*"event" + 0.005*"new" + 0.004*"black" + 0.004*"body" + 0.004*"white" + 0.004*"design" + 0.004*"world" + 0.003*"muscle"
0.005*"arm" + 0.005*"shirt" + 0.005*"t" + 0.005*"neck" + 0.005*"body" + 0.004*"world" + 0.004*"entertainment" + 0.004*"outdoor" + 0.004*"nature" + 0.004*"in"

------------------------LDA with TFIDF ----------------------- 

0.005*"new" + 0.004*"game" + 0.004*"short" + 0.004*"plant" + 0.004*"outdoor" + 0.004*"world" + 0.004*"shorts" + 0.004*"neck" + 0.004*"body" + 0.004*"shirt"
0.005*"shirt" + 0.005*"body" + 0.005*"--" + 0.004*"outdoor" + 0.004*"forehead" + 0.004*"neck" + 0.004*"world" + 0.004*"available" + 0.004*"electric" + 0.004*"eyebrow"
0.005*"white" + 0.005*"event" + 0.004*"entertainment" + 0.004*"nature" + 0.004*"joint" + 0.004*"in" + 0.004*"black" + 0.004*"design" + 0.004

0.007*"body" + 0.006*"create" + 0.006*"--" + 0.006*"blue" + 0.005*"knee" + 0.005*"outdoor" + 0.004*"arm" + 0.004*"joint" + 0.004*"black" + 0.004*"uniform"
0.009*"happy" + 0.006*"people" + 0.006*"uniform" + 0.006*"new" + 0.005*"nature" + 0.005*"player" + 0.005*"white" + 0.005*"in" + 0.004*"short" + 0.004*"and"
0.006*"create" + 0.006*"shirt" + 0.006*"player" + 0.005*"t" + 0.005*"shorts" + 0.005*"make" + 0.005*"event" + 0.005*"design" + 0.005*"blue" + 0.004*"knee"
# Topics 3 Alpha 0.7 ETA val None top_n 30

------------------------LDA without TFIDF ----------------------- 

0.005*"shorts" + 0.005*"white" + 0.005*"event" + 0.004*"and" + 0.004*"new" + 0.004*"plant" + 0.004*"leisure" + 0.004*"design" + 0.004*"body" + 0.004*"link"
0.005*"body" + 0.005*"black" + 0.005*"shirt" + 0.005*"--" + 0.004*"neck" + 0.004*"forehead" + 0.004*"head" + 0.004*"design" + 0.004*"new" + 0.004*"entertainment"
0.005*"world" + 0.005*"outdoor" + 0.005*"neck" + 0.005*"nature" + 0.004*"shirt" + 0.004*"electric" + 0.0

0.005*"shirt" + 0.005*"knee" + 0.004*"outdoor" + 0.004*"player" + 0.004*"design" + 0.004*"people" + 0.004*"uniform" + 0.004*"black" + 0.004*"happy" + 0.004*"blue"
0.006*"create" + 0.006*"uniform" + 0.005*"body" + 0.005*"happy" + 0.004*"blue" + 0.004*"world" + 0.004*"player" + 0.004*"neck" + 0.003*"shorts" + 0.003*"joint"
0.006*"create" + 0.005*"happy" + 0.004*"people" + 0.004*"blue" + 0.004*"see" + 0.004*"make" + 0.003*"white" + 0.003*"plant" + 0.003*"in" + 0.003*"cloud"

------------------------LDA with TFIDF ----------------------- 

0.006*"create" + 0.006*"uniform" + 0.005*"body" + 0.005*"happy" + 0.005*"people" + 0.004*"in" + 0.004*"blue" + 0.004*"new" + 0.004*"plant" + 0.004*"black"
0.005*"blue" + 0.004*"happy" + 0.004*"electric" + 0.004*"knee" + 0.004*"people" + 0.004*"neck" + 0.003*"event" + 0.003*"new" + 0.003*"create" + 0.003*"design"
0.006*"player" + 0.005*"create" + 0.004*"design" + 0.004*"make" + 0.004*"happy" + 0.004*"jersey" + 0.004*"outdoor" + 0.004*"knee" + 0.004*"short

0.006*"and" + 0.006*"shirt" + 0.005*"new" + 0.005*"body" + 0.005*"t" + 0.004*"run" + 0.004*"--" + 0.004*"shorts" + 0.004*"cloud" + 0.004*"outerwear"
0.007*"world" + 0.005*"eyebrow" + 0.005*"shorts" + 0.005*"neck" + 0.004*"black" + 0.004*"nature" + 0.004*"shirt" + 0.004*"change" + 0.004*"leisure" + 0.004*"arm"
0.006*"neck" + 0.006*"design" + 0.005*"shorts" + 0.005*"black" + 0.004*"new" + 0.004*"outdoor" + 0.004*"--" + 0.004*"hair" + 0.004*"waist" + 0.004*"week"
0.006*"in" + 0.006*"nature" + 0.005*"outdoor" + 0.005*"leisure" + 0.005*"new" + 0.005*"plant" + 0.005*"design" + 0.004*"world" + 0.004*"recreation" + 0.004*"event"
0.006*"black" + 0.006*"white" + 0.005*"electric" + 0.005*"shirt" + 0.005*"entertainment" + 0.005*"body" + 0.005*"outdoor" + 0.004*"plant" + 0.004*"design" + 0.004*"adidasoriginal"
# Topics 5 Alpha 0.5 ETA val 0.5 top_n None

------------------------LDA without TFIDF ----------------------- 

0.012*"shoe" + 0.009*"photography" + 0.008*"flash" + 0.008*"art" + 0.006*"font

...

# Topics 10 Alpha 0.7 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.016*"shoe" + 0.011*"sports" + 0.009*"sportswear" + 0.009*"equipment" + 0.008*"font" + 0.008*"create" + 0.007*"street" + 0.006*"footwear" + 0.006*"sky" + 0.006*"uniform"
0.012*"shoe" + 0.009*"sports" + 0.008*"sport" + 0.008*"sportswear" + 0.008*"equipment" + 0.007*"heretocreate" + 0.007*"photography" + 0.007*"fashion" + 0.007*"player" + 0.007*"street"
0.010*"fashion" + 0.010*"shoe" + 0.008*"sports" + 0.007*"font" + 0.007*"flash" + 0.007*"photography" + 0.007*"people" + 0.006*"black" + 0.006*"human" + 0.006*"equipment"
0.027*"shoe" + 0.010*"equipment" + 0.009*"fashion" + 0.009*"sportswear" + 0.007*"footwear" + 0.007*"sports" + 0.007*"flash" + 0.007*"photography" + 0.006*"-" + 0.006*"sleeve"
0.014*"shoe" + 0.008*"sky" + 0.008*"sports" + 0.008*"leg" + 0.008*"thigh" + 0.007*"fashion" + 0.007*"sportswear" + 0.007*"flash" + 0.007*"photography" + 0.007*"-"
0.011*"shoe" + 0.009*"


------------------------LDA with TFIDF ----------------------- 

0.007*"player" + 0.006*"blue" + 0.006*"people" + 0.005*"shorts" + 0.005*"nature" + 0.005*"uniform" + 0.004*"electric" + 0.004*"create" + 0.004*"new" + 0.004*"and"
0.006*"create" + 0.005*"happy" + 0.005*"make" + 0.005*"people" + 0.005*"white" + 0.004*"waist" + 0.004*"and" + 0.004*"electric" + 0.004*"blue" + 0.004*"knee"
0.007*"create" + 0.006*"knee" + 0.005*"body" + 0.005*"neck" + 0.005*"outdoor" + 0.005*"people" + 0.005*"happy" + 0.004*"black" + 0.004*"jersey" + 0.004*"make"
0.007*"create" + 0.006*"make" + 0.006*"happy" + 0.006*"people" + 0.005*"uniform" + 0.005*"body" + 0.005*"world" + 0.005*"new" + 0.004*"player" + 0.004*"nature"
0.008*"blue" + 0.006*"happy" + 0.005*"knee" + 0.005*"nature" + 0.005*"create" + 0.005*"outdoor" + 0.004*"electric" + 0.004*"neck" + 0.004*"see" + 0.004*"shirt"
0.007*"make" + 0.006*"happy" + 0.005*"world" + 0.005*"create" + 0.005*"shirt" + 0.005*"outdoor" + 0.005*"player" + 0.005*"knee" + 0.00

# Topics 10 Alpha 0.7 ETA val 0.5 top_n 10

------------------------LDA without TFIDF ----------------------- 

0.005*"sky" + 0.004*"leg" + 0.004*"footwear" + 0.004*"gesture" + 0.003*"thigh" + 0.003*"human" + 0.003*"art" + 0.003*"design" + 0.003*"create" + 0.003*"and"
0.004*"create" + 0.004*"gesture" + 0.004*"-" + 0.004*"leg" + 0.003*"shirt" + 0.003*"sky" + 0.003*"thigh" + 0.003*"and" + 0.003*"make" + 0.003*"street"
0.005*"footwear" + 0.004*"thigh" + 0.004*"gesture" + 0.004*"leg" + 0.004*"street" + 0.004*"human" + 0.004*"art" + 0.003*"heretocreate" + 0.003*"create" + 0.003*"sky"
0.005*"heretocreate" + 0.004*"footwear" + 0.004*"sky" + 0.004*"street" + 0.004*"gesture" + 0.004*"thigh" + 0.004*"happy" + 0.003*"neck" + 0.003*"body" + 0.003*"blue"
0.006*"leg" + 0.004*"thigh" + 0.004*"happy" + 0.004*"street" + 0.004*"knee" + 0.004*"human" + 0.003*"sky" + 0.003*"art" + 0.003*"create" + 0.003*"heretocreate"
0.005*"footwear" + 0.005*"-" + 0.004*"create" + 0.004*"gesture" + 0.004*"thigh" + 0.004*


------------------------LDA with TFIDF ----------------------- 

0.003*"event" + 0.003*"world" + 0.003*"body" + 0.002*"adidasoriginal" + 0.002*"electric" + 0.002*"shirt" + 0.002*"muscle" + 0.002*"water" + 0.002*"outdoor" + 0.002*"design"
0.004*"event" + 0.004*"world" + 0.003*"entertainment" + 0.003*"--" + 0.003*"electric" + 0.003*"outdoor" + 0.003*"shorts" + 0.003*"in" + 0.002*"black" + 0.002*"t"
0.004*"body" + 0.003*"shirt" + 0.003*"nature" + 0.003*"white" + 0.003*"new" + 0.003*"plant" + 0.003*"in" + 0.002*"t" + 0.002*"world" + 0.002*"and"
0.003*"new" + 0.002*"design" + 0.002*"outdoor" + 0.002*"smile" + 0.002*"black" + 0.002*"body" + 0.002*"arm" + 0.002*"shirt" + 0.002*"event" + 0.002*"world"
0.003*"shirt" + 0.003*"shorts" + 0.003*"plant" + 0.003*"cloud" + 0.003*"black" + 0.003*"white" + 0.003*"and" + 0.003*"adidasoriginal" + 0.002*"outdoor" + 0.002*"available"
0.003*"nature" + 0.003*"neck" + 0.003*"outdoor" + 0.003*"electric" + 0.003*"plant" + 0.003*"arm" + 0.003*"leisure" + 0.003*"

# Topics 10 Alpha 0.7 ETA val 0.7 top_n 20

------------------------LDA without TFIDF ----------------------- 

0.003*"create" + 0.003*"happy" + 0.003*"make" + 0.002*"shirt" + 0.002*"knee" + 0.002*"player" + 0.002*"neck" + 0.002*"new" + 0.002*"black" + 0.002*"uniform"
0.003*"happy" + 0.003*"blue" + 0.003*"create" + 0.003*"make" + 0.003*"player" + 0.002*"uniform" + 0.002*"people" + 0.002*"outdoor" + 0.002*"arm" + 0.002*"event"
0.003*"knee" + 0.003*"uniform" + 0.003*"create" + 0.003*"player" + 0.003*"happy" + 0.002*"blue" + 0.002*"design" + 0.002*"shorts" + 0.002*"in" + 0.002*"body"
0.004*"uniform" + 0.003*"body" + 0.003*"blue" + 0.003*"electric" + 0.003*"create" + 0.002*"shorts" + 0.002*"--" + 0.002*"knee" + 0.002*"see" + 0.002*"nature"
0.003*"create" + 0.002*"knee" + 0.002*"new" + 0.002*"design" + 0.002*"eyebrow" + 0.002*"shorts" + 0.002*"neck" + 0.002*"blue" + 0.002*"shirt" + 0.002*"uniform"
0.003*"player" + 0.003*"world" + 0.003*"knee" + 0.002*"create" + 0.002*"jersey" + 0.002*"shirt

# Nike

## Only Captions

In [14]:
data = pd.read_csv("./Nike/nike_caption_tf.csv", index_col=0)
print(data.columns)
column_name = 'caption_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['really', 'shawnjohnsons', 'toddantonyphoto', 'nosajthe', 'broken',
       'tired', 'collect', 'brasileira', 'best', 'happiness',
       ...
       'universal', 'narrow', 'bowerman', 'corner', 'rise', 'tomckean',
       'fencer', 'gomofarah', 'racer', 'caption_list'],
      dtype='object', length=2832)


In [16]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('nike_topic_model_caption_tf.xlsx')
writer_tfidf = pd.ExcelWriter('nike_topic_model_caption_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.024*"justdoit" + 0.014*"``" + 0.010*"nike" + 0.008*"nt" + 0.007*"get" + 0.007*"run" + 0.006*"go" + 0.006*"start" + 0.006*"world" + 0.006*"day"
0.023*"justdoit" + 0.010*"nike" + 0.007*"n't" + 0.007*"run" + 0.007*"``" + 0.007*"get" + 0.007*"make" + 0.005*"good" + 0.005*"'s" + 0.005*"woman"
0.018*"``" + 0.013*"justdoit" + 0.010*"world" + 0.010*"n't" + 0.008*"like" + 0.008*"nike" + 0.008*"run" + 0.007*"'s" + 0.005*"game" + 0.005*"change"

------------------------LDA with TFIDF ----------------------- 

0.018*"``" + 0.017*"justdoit" + 0.011*"nike" + 0.010*"n't" + 0.009*"world" + 0.007*"'s" + 0.007*"run" + 0.006*"never" + 0.005*"new" + 0.005*"change"
0.017*"justdoit" + 0.014*"nike" + 0.007*"run" + 0.007*"make" + 0.007*"n't" + 0.006*"get" + 0.006*"``" + 0.005*"go" + 0.005*"good" + 0.005*"new"
0.026*"justdoit" + 0.014*"``" + 0.010*"get" + 0.008*"run" + 0.006*"n't" + 0.006*"like" 

0.003*"well" + 0.003*"always" + 0.003*"bio" + 0.003*"inspire" + 0.003*"athlete" + 0.003*"link" + 0.003*"show" + 0.003*"back" + 0.003*"love" + 0.003*"thing"
0.004*"find" + 0.003*"way" + 0.003*"nike+" + 0.003*"every" + 0.003*"show" + 0.003*"look" + 0.003*"tell" + 0.003*"nikeplus" + 0.003*"earn" + 0.003*"move"
0.004*"today" + 0.004*"big" + 0.004*"dream" + 0.004*"keep" + 0.003*"look" + 0.003*"inspire" + 0.003*"celebrate" + 0.003*"work" + 0.003*"every" + 0.003*"way"
# Topics 3 Alpha 0.5 ETA val 0.7 top_n None

------------------------LDA without TFIDF ----------------------- 

0.013*"``" + 0.013*"justdoit" + 0.011*"n't" + 0.006*"run" + 0.005*"get" + 0.005*"time" + 0.005*"like" + 0.005*"nike" + 0.004*"'s" + 0.004*"world"
0.016*"justdoit" + 0.011*"``" + 0.008*"get" + 0.008*"nike" + 0.006*"world" + 0.005*"n't" + 0.005*"go" + 0.004*"good" + 0.004*"'s" + 0.004*"new"
0.020*"justdoit" + 0.010*"nike" + 0.009*"run" + 0.008*"``" + 0.006*"world" + 0.005*"sport" + 0.005*"'s" + 0.005*"nt" + 0.004*"make"

0.005*"inspire" + 0.004*"way" + 0.003*"back" + 0.003*"always" + 0.003*"athlete" + 0.003*"tell" + 0.003*"home" + 0.003*"find" + 0.003*"every" + 0.003*"keep"
0.005*"every" + 0.005*"show" + 0.004*"keep" + 0.004*"nikeplus" + 0.004*"dream" + 0.003*"know" + 0.003*"look" + 0.003*"big" + 0.003*"nikefuel" + 0.003*"athlete"
0.004*"today" + 0.004*"find" + 0.003*"look" + 0.003*"well" + 0.003*"way" + 0.003*"finish" + 0.003*"work" + 0.003*"know" + 0.003*"tell" + 0.003*"dream"

------------------------LDA with TFIDF ----------------------- 

0.004*"keep" + 0.003*"every" + 0.003*"today" + 0.003*"dream" + 0.003*"look" + 0.003*"inspire" + 0.003*"tell" + 0.003*"share" + 0.003*"life" + 0.003*"next"
0.004*"find" + 0.004*"work" + 0.004*"way" + 0.003*"look" + 0.003*"know" + 0.003*"today" + 0.003*"player" + 0.003*"inspire" + 0.003*"athlete" + 0.003*"every"
0.004*"keep" + 0.004*"dream" + 0.004*"show" + 0.003*"way" + 0.003*"nikeplus" + 0.003*"find" + 0.003*"inspire" + 0.003*"move" + 0.003*"know" + 0.003*"look"


0.004*"today" + 0.003*"community" + 0.003*"inspire" + 0.003*"look" + 0.003*"big" + 0.003*"show" + 0.003*"celebrate" + 0.003*"find" + 0.003*"nikefuel" + 0.003*"back"
0.003*"dream" + 0.003*"way" + 0.003*"know" + 0.003*"tell" + 0.002*"athlete" + 0.002*"well" + 0.002*"player" + 0.002*"quot" + 0.002*"find" + 0.002*"keep"
0.003*"every" + 0.003*"keep" + 0.003*"finish" + 0.003*"find" + 0.003*"show" + 0.002*"work" + 0.002*"nikeplus" + 0.002*"athlete" + 0.002*"look" + 0.002*"inspire"

------------------------LDA with TFIDF ----------------------- 

0.003*"find" + 0.003*"big" + 0.003*"today" + 0.003*"every" + 0.003*"athlete" + 0.003*"look" + 0.003*"keep" + 0.002*"dream" + 0.002*"inspire" + 0.002*"nikeplus"
0.003*"know" + 0.003*"inspire" + 0.003*"keep" + 0.003*"community" + 0.003*"give" + 0.003*"dream" + 0.003*"become" + 0.003*"work" + 0.003*"look" + 0.002*"today"
0.003*"way" + 0.003*"next" + 0.002*"fast" + 0.002*"nikeplus" + 0.002*"every" + 0.002*"look" + 0.002*"another" + 0.002*"well" + 0.002*"f

0.014*"justdoit" + 0.009*"``" + 0.007*"nike" + 0.007*"run" + 0.007*"world" + 0.006*"n't" + 0.005*"'s" + 0.005*"make" + 0.005*"go" + 0.005*"change"
0.016*"justdoit" + 0.007*"``" + 0.005*"n't" + 0.005*"never" + 0.005*"go" + 0.004*"nike" + 0.004*"run" + 0.004*"like" + 0.004*"new" + 0.004*"play"
0.015*"justdoit" + 0.009*"get" + 0.008*"nike" + 0.007*"n't" + 0.006*"world" + 0.006*"good" + 0.005*"run" + 0.005*"``" + 0.004*"make" + 0.004*"play"
0.019*"justdoit" + 0.018*"``" + 0.006*"nike" + 0.006*"run" + 0.005*"n't" + 0.005*"world" + 0.004*"see" + 0.004*"quot" + 0.004*"year" + 0.004*"'s"
0.012*"justdoit" + 0.011*"``" + 0.010*"nike" + 0.007*"nt" + 0.005*"'s" + 0.005*"run" + 0.005*"n't" + 0.005*"get" + 0.005*"make" + 0.004*"sport"
# Topics 5 Alpha 0.5 ETA val 0.5 top_n 10

------------------------LDA without TFIDF ----------------------- 

0.007*"play" + 0.005*"like" + 0.004*"go" + 0.003*"start" + 0.003*"big" + 0.003*"football" + 0.003*"find" + 0.003*"sport" + 0.003*"nt" + 0.003*"way"
0.005*"goo

...

# Topics 10 Alpha 0.7 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.020*"justdoit" + 0.012*"``" + 0.009*"n't" + 0.008*"nike" + 0.007*"get" + 0.006*"world" + 0.006*"go" + 0.006*"new" + 0.005*"game" + 0.004*"find"
0.020*"justdoit" + 0.015*"nike" + 0.012*"``" + 0.009*"run" + 0.008*"world" + 0.007*"get" + 0.006*"'s" + 0.005*"n't" + 0.005*"see" + 0.005*"change"
0.018*"justdoit" + 0.014*"nike" + 0.011*"``" + 0.009*"run" + 0.008*"n't" + 0.005*"get" + 0.005*"new" + 0.005*"like" + 0.005*"world" + 0.004*"make"
0.023*"justdoit" + 0.018*"``" + 0.010*"n't" + 0.006*"nike" + 0.006*"make" + 0.006*"'s" + 0.006*"run" + 0.006*"go" + 0.005*"nt" + 0.005*"get"
0.024*"``" + 0.015*"justdoit" + 0.009*"run" + 0.008*"world" + 0.008*"n't" + 0.008*"nike" + 0.007*"'s" + 0.006*"play" + 0.005*"inspire" + 0.005*"good"
0.019*"justdoit" + 0.010*"``" + 0.008*"nike" + 0.008*"run" + 0.007*"get" + 0.007*"world" + 0.006*"n't" + 0.005*"make" + 0.004*"'s" + 0.004*"new"
0.018*"ju

# Topics 10 Alpha 0.7 ETA val None top_n 30

------------------------LDA without TFIDF ----------------------- 

0.004*"every" + 0.004*"find" + 0.003*"dream" + 0.003*"inspire" + 0.003*"athlete" + 0.003*"keep" + 0.003*"work" + 0.003*"way" + 0.003*"love" + 0.003*"today"
0.004*"today" + 0.004*"show" + 0.003*"dream" + 0.003*"look" + 0.003*"athlete" + 0.003*"next" + 0.003*"back" + 0.003*"thing" + 0.003*"let" + 0.003*"give"
0.004*"inspire" + 0.004*"way" + 0.004*"keep" + 0.004*"instagram" + 0.004*"dream" + 0.004*"today" + 0.004*"find" + 0.004*"big" + 0.003*"continue" + 0.003*"celebrate"
0.004*"community" + 0.004*"inspire" + 0.003*"look" + 0.003*"way" + 0.003*"show" + 0.003*"two" + 0.003*"big" + 0.003*"back" + 0.003*"athlete" + 0.003*"life"
0.004*"know" + 0.004*"inspire" + 0.004*"way" + 0.004*"today" + 0.004*"every" + 0.004*"find" + 0.003*"athlete" + 0.003*"share" + 0.003*"dream" + 0.003*"keep"
0.004*"look" + 0.004*"tell" + 0.004*"inspire" + 0.003*"nikeplus" + 0.003*"well" + 0.003*"work" + 0.0

# Topics 10 Alpha 0.7 ETA val 0.5 top_n 20

------------------------LDA without TFIDF ----------------------- 

0.002*"first" + 0.002*"change" + 0.002*"find" + 0.002*"year" + 0.002*"never" + 0.002*"today" + 0.002*"know" + 0.002*"look" + 0.002*"nikeplus" + 0.002*"earn"
0.003*"keep" + 0.002*"community" + 0.002*"see" + 0.002*"inspire" + 0.002*"year" + 0.002*"today" + 0.002*"woman" + 0.002*"celebrate" + 0.002*"work" + 0.002*"every"
0.002*"look" + 0.002*"change" + 0.002*"well" + 0.002*"inspire" + 0.002*"every" + 0.002*"keep" + 0.002*"win" + 0.002*"tell" + 0.002*"football" + 0.002*"way"
0.003*"marathon" + 0.002*"athlete" + 0.002*"first" + 0.002*"win" + 0.002*"find" + 0.002*"bio" + 0.002*"team" + 0.002*"become" + 0.002*"today" + 0.002*"year"
0.002*"woman" + 0.002*"win" + 0.002*"team" + 0.002*"show" + 0.002*"football" + 0.002*"year" + 0.002*"quot" + 0.002*"first" + 0.002*"see" + 0.002*"work"
0.003*"first" + 0.002*"way" + 0.002*"today" + 0.002*"change" + 0.002*"see" + 0.002*"look" + 0.002*"ever


------------------------LDA with TFIDF ----------------------- 

0.008*"justdoit" + 0.006*"``" + 0.005*"run" + 0.004*"nike" + 0.003*"'s" + 0.003*"get" + 0.003*"world" + 0.002*"make" + 0.002*"n't" + 0.002*"new"
0.008*"justdoit" + 0.005*"``" + 0.005*"n't" + 0.005*"nike" + 0.003*"run" + 0.003*"'s" + 0.003*"world" + 0.002*"make" + 0.002*"time" + 0.002*"good"
0.009*"justdoit" + 0.005*"``" + 0.004*"nike" + 0.003*"n't" + 0.003*"'s" + 0.003*"world" + 0.002*"new" + 0.002*"run" + 0.002*"good" + 0.002*"sport"
0.011*"justdoit" + 0.006*"nike" + 0.006*"``" + 0.005*"n't" + 0.004*"world" + 0.003*"nt" + 0.003*"run" + 0.003*"like" + 0.003*"get" + 0.003*"sport"
0.010*"justdoit" + 0.007*"``" + 0.004*"n't" + 0.004*"get" + 0.004*"run" + 0.003*"world" + 0.003*"nike" + 0.003*"go" + 0.003*"make" + 0.003*"new"
0.008*"justdoit" + 0.005*"``" + 0.003*"n't" + 0.003*"nike" + 0.003*"get" + 0.003*"play" + 0.003*"'s" + 0.003*"run" + 0.003*"new" + 0.003*"good"
0.008*"justdoit" + 0.008*"``" + 0.004*"run" + 0.004*"get" +


------------------------LDA with TFIDF ----------------------- 

0.002*"inspire" + 0.001*"next" + 0.001*"show" + 0.001*"nikefuel" + 0.001*"well" + 0.001*"work" + 0.001*"home" + 0.001*"find" + 0.001*"know" + 0.001*"every"
0.002*"today" + 0.001*"dream" + 0.001*"always" + 0.001*"show" + 0.001*"move" + 0.001*"keep" + 0.001*"instagram" + 0.001*"photo" + 0.001*"help" + 0.001*"back"
0.002*"show" + 0.002*"find" + 0.002*"inspire" + 0.002*"dream" + 0.002*"every" + 0.001*"keep" + 0.001*"community" + 0.001*"today" + 0.001*"look" + 0.001*"photo"
0.002*"look" + 0.002*"every" + 0.002*"inspire" + 0.002*"way" + 0.002*"work" + 0.002*"keep" + 0.001*"big" + 0.001*"back" + 0.001*"know" + 0.001*"nikeplus"
0.002*"know" + 0.002*"way" + 0.002*"keep" + 0.001*"tag" + 0.001*"look" + 0.001*"today" + 0.001*"nike+" + 0.001*"tell" + 0.001*"inspire" + 0.001*"every"
0.002*"find" + 0.002*"way" + 0.002*"look" + 0.001*"today" + 0.001*"community" + 0.001*"inspire" + 0.001*"know" + 0.001*"athlete" + 0.001*"every" + 0.001*"

## Only Image Labels

In [17]:
data = pd.read_csv("./Nike/nike_label_tf.csv", index_col=0)
print(data.columns)
column_name = 'labels_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['fog', 'engineering', 'lawn', 'needle', 'fixture', 'active', 'blue',
       'sea', 'appliance', 'drawer',
       ...
       'net', 'branch', 'skateboarder', 'circle', 'fashion', 'boat', 'cricket',
       'skyscraper', 'labels', 'labels_list'],
      dtype='object', length=673)


In [18]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('nike_topic_model_image_tf.xlsx')
writer_tfidf = pd.ExcelWriter('nike_topic_model_image_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.029*"shoe" + 0.023*"equipment" + 0.023*"sports" + 0.022*"sky" + 0.017*"photography" + 0.017*"football" + 0.014*"flash" + 0.013*"cloud" + 0.013*"sport" + 0.012*"outdoor"
0.032*"sky" + 0.024*"people" + 0.022*"in" + 0.022*"nature" + 0.017*"sports" + 0.017*"happy" + 0.017*"plant" + 0.016*"equipment" + 0.016*"water" + 0.016*"shoe"
0.041*"sports" + 0.028*"equipment" + 0.026*"shoe" + 0.019*"sport" + 0.018*"player" + 0.018*"sportswear" + 0.016*"sky" + 0.015*"gesture" + 0.015*"leg" + 0.014*"uniform"

------------------------LDA with TFIDF ----------------------- 

0.045*"shoe" + 0.025*"sky" + 0.018*"footwear" + 0.016*"nature" + 0.015*"leg" + 0.014*"sportswear" + 0.013*"street" + 0.012*"photography" + 0.012*"sports" + 0.012*"flash"
0.031*"sports" + 0.031*"equipment" + 0.027*"sky" + 0.019*"people" + 0.019*"happy" + 0.018*"in" + 0.018*"nature" + 0.017*"plant" + 0.015*"sport" + 0.014*

0.017*"landscape" + 0.017*"grass" + 0.015*"mountain" + 0.014*"outdoor" + 0.013*"font" + 0.013*"natural" + 0.012*"shorts" + 0.011*"athletic" + 0.011*"art" + 0.011*"slope"
0.015*"street" + 0.014*"entertainment" + 0.013*"short" + 0.012*"gear" + 0.011*"art" + 0.011*"shirt" + 0.011*"t" + 0.011*"smile" + 0.010*"white" + 0.010*"font"
0.015*"human" + 0.014*"shirt" + 0.014*"light" + 0.014*"smile" + 0.013*"knee" + 0.012*"elbow" + 0.012*"hand" + 0.012*"gear" + 0.011*"outdoor" + 0.011*"street"

------------------------LDA with TFIDF ----------------------- 

0.016*"shorts" + 0.016*"shirt" + 0.015*"smile" + 0.014*"landscape" + 0.012*"street" + 0.012*"t" + 0.011*"mountain" + 0.011*"blue" + 0.010*"art" + 0.010*"elbow"
0.016*"outdoor" + 0.013*"grass" + 0.013*"font" + 0.013*"shorts" + 0.013*"arm" + 0.013*"light" + 0.012*"design" + 0.012*"body" + 0.012*"athletic" + 0.012*"blue"
0.017*"human" + 0.015*"short" + 0.013*"gear" + 0.013*"knee" + 0.012*"wood" + 0.012*"art" + 0.011*"street" + 0.011*"body" + 0.01

0.016*"thigh" + 0.015*"grass" + 0.015*"recreation" + 0.014*"tree" + 0.014*"fashion" + 0.012*"human" + 0.012*"body" + 0.012*"leisure" + 0.011*"street" + 0.011*"art"
0.015*"fashion" + 0.014*"gear" + 0.013*"street" + 0.012*"shorts" + 0.012*"tree" + 0.012*"human" + 0.012*"sleeve" + 0.012*"thigh" + 0.012*"font" + 0.011*"blue"
0.016*"-" + 0.014*"smile" + 0.013*"sleeve" + 0.013*"water" + 0.013*"leisure" + 0.012*"outdoor" + 0.012*"wood" + 0.012*"light" + 0.011*"and" + 0.011*"tree"
# Topics 3 Alpha 0.7 ETA val None top_n 30

------------------------LDA without TFIDF ----------------------- 

0.013*"landscape" + 0.013*"light" + 0.013*"shorts" + 0.012*"art" + 0.012*"gear" + 0.012*"human" + 0.012*"knee" + 0.012*"street" + 0.012*"smile" + 0.011*"outdoor"
0.015*"street" + 0.013*"blue" + 0.012*"font" + 0.012*"human" + 0.012*"event" + 0.012*"shorts" + 0.011*"knee" + 0.011*"grass" + 0.010*"wood" + 0.010*"art"
0.016*"outdoor" + 0.014*"shirt" + 0.013*"body" + 0.013*"elbow" + 0.013*"muscle" + 0.012*"human

0.014*"fashion" + 0.014*"street" + 0.012*"thigh" + 0.012*"human" + 0.011*"sleeve" + 0.009*"shirt" + 0.009*"grass" + 0.009*"-" + 0.009*"white" + 0.009*"knee"
0.013*"thigh" + 0.013*"sleeve" + 0.012*"fashion" + 0.011*"tree" + 0.011*"outdoor" + 0.011*"short" + 0.011*"landscape" + 0.011*"muscle" + 0.011*"font" + 0.011*"leisure"
0.016*"tree" + 0.015*"and" + 0.015*"recreation" + 0.012*"leisure" + 0.011*"-" + 0.011*"shorts" + 0.011*"water" + 0.010*"light" + 0.010*"art" + 0.009*"blue"

------------------------LDA with TFIDF ----------------------- 

0.013*"sleeve" + 0.012*"art" + 0.012*"-" + 0.011*"shorts" + 0.011*"fashion" + 0.011*"thigh" + 0.011*"white" + 0.011*"smile" + 0.011*"grass" + 0.010*"tree"
0.013*"human" + 0.013*"fashion" + 0.012*"leisure" + 0.012*"thigh" + 0.012*"body" + 0.011*"elbow" + 0.011*"and" + 0.010*"water" + 0.010*"shirt" + 0.010*"-"
0.016*"outdoor" + 0.015*"tree" + 0.013*"recreation" + 0.013*"water" + 0.013*"street" + 0.012*"font" + 0.011*"thigh" + 0.011*"leisure" + 0.011*"

0.021*"art" + 0.017*"outdoor" + 0.015*"entertainment" + 0.014*"event" + 0.013*"shirt" + 0.012*"shorts" + 0.011*"mountain" + 0.011*"font" + 0.011*"arm" + 0.010*"human"
0.023*"shorts" + 0.020*"smile" + 0.019*"sneakers" + 0.017*"knee" + 0.016*"short" + 0.016*"outdoor" + 0.014*"muscle" + 0.013*"human" + 0.013*"grass" + 0.013*"body"
0.029*"street" + 0.015*"wood" + 0.014*"human" + 0.014*"body" + 0.013*"white" + 0.013*"shirt" + 0.013*"knee" + 0.012*"gear" + 0.012*"road" + 0.012*"light"
0.018*"landscape" + 0.015*"human" + 0.015*"light" + 0.012*"natural" + 0.012*"mountain" + 0.011*"design" + 0.010*"slope" + 0.010*"art" + 0.010*"short" + 0.010*"outdoor"
0.020*"grass" + 0.017*"blue" + 0.014*"gear" + 0.013*"font" + 0.013*"knee" + 0.012*"elbow" + 0.011*"electric" + 0.011*"building" + 0.010*"design" + 0.009*"neck"
# Topics 5 Alpha 0.5 ETA val 0.5 top_n None

------------------------LDA without TFIDF ----------------------- 

0.024*"equipment" + 0.022*"sky" + 0.018*"sports" + 0.017*"shoe" + 0.016*"na

...

0.037*"sports" + 0.034*"equipment" + 0.026*"sky" + 0.020*"sportswear" + 0.019*"uniform" + 0.015*"player" + 0.015*"nature" + 0.014*"jersey" + 0.013*"sport" + 0.013*"happy"
0.036*"shoe" + 0.027*"sports" + 0.024*"sky" + 0.019*"equipment" + 0.018*"footwear" + 0.014*"nature" + 0.013*"in" + 0.013*"water" + 0.012*"leg" + 0.012*"people"
0.022*"equipment" + 0.021*"sports" + 0.020*"sky" + 0.016*"people" + 0.016*"sport" + 0.015*"player" + 0.014*"gesture" + 0.013*"happy" + 0.013*"nature" + 0.013*"in"
0.039*"sports" + 0.025*"sky" + 0.020*"shoe" + 0.016*"player" + 0.016*"sportswear" + 0.014*"equipment" + 0.014*"sport" + 0.014*"nature" + 0.012*"flash" + 0.012*"gesture"
0.027*"sky" + 0.025*"shoe" + 0.023*"equipment" + 0.023*"sports" + 0.020*"cloud" + 0.014*"nature" + 0.013*"footwear" + 0.012*"happy" + 0.011*"in" + 0.011*"leg"
0.023*"sports" + 0.021*"shoe" + 0.020*"equipment" + 0.019*"sky" + 0.015*"people" + 0.014*"sport" + 0.014*"plant" + 0.013*"nature" + 0.013*"uniform" + 0.013*"human"
0.036*"shoe" +

0.014*"fashion" + 0.014*"thigh" + 0.012*"and" + 0.011*"recreation" + 0.011*"shirt" + 0.011*"sleeve" + 0.011*"muscle" + 0.011*"human" + 0.011*"shorts" + 0.011*"mountain"
0.013*"font" + 0.013*"grass" + 0.013*"thigh" + 0.013*"sleeve" + 0.011*"fashion" + 0.011*"street" + 0.011*"water" + 0.011*"leisure" + 0.011*"and" + 0.010*"tree"
0.014*"tree" + 0.014*"sleeve" + 0.014*"thigh" + 0.013*"leisure" + 0.013*"outdoor" + 0.013*"recreation" + 0.012*"and" + 0.011*"grass" + 0.011*"-" + 0.011*"landscape"
0.015*"and" + 0.013*"shorts" + 0.013*"thigh" + 0.013*"art" + 0.013*"water" + 0.013*"tree" + 0.013*"-" + 0.012*"sleeve" + 0.011*"elbow" + 0.011*"human"
0.015*"fashion" + 0.015*"sleeve" + 0.014*"knee" + 0.014*"tree" + 0.013*"street" + 0.012*"thigh" + 0.011*"water" + 0.011*"gear" + 0.010*"body" + 0.010*"outdoor"
0.017*"fashion" + 0.013*"sleeve" + 0.013*"thigh" + 0.013*"water" + 0.011*"font" + 0.011*"gear" + 0.011*"tree" + 0.010*"recreation" + 0.010*"street" + 0.010*"human"
0.015*"fashion" + 0.014*"human"

0.011*"cloud" + 0.011*"sleeve" + 0.010*"fashion" + 0.010*"flash" + 0.009*"photography" + 0.009*"uniform" + 0.009*"footwear" + 0.009*"happy" + 0.008*"player" + 0.008*"knee"
0.014*"photography" + 0.013*"flash" + 0.013*"uniform" + 0.012*"happy" + 0.011*"leg" + 0.011*"street" + 0.010*"footwear" + 0.010*"plant" + 0.010*"player" + 0.010*"cloud"
0.015*"cloud" + 0.013*"thigh" + 0.012*"happy" + 0.011*"plant" + 0.010*"footwear" + 0.010*"uniform" + 0.009*"photography" + 0.009*"human" + 0.009*"flash" + 0.009*"player"
0.013*"footwear" + 0.011*"happy" + 0.011*"plant" + 0.010*"cloud" + 0.010*"flash" + 0.009*"leisure" + 0.009*"player" + 0.009*"fashion" + 0.009*"tree" + 0.008*"uniform"
0.013*"uniform" + 0.012*"leg" + 0.011*"player" + 0.011*"footwear" + 0.010*"thigh" + 0.009*"jersey" + 0.009*"tree" + 0.009*"happy" + 0.008*"plant" + 0.008*"cloud"
0.012*"player" + 0.011*"sleeve" + 0.011*"happy" + 0.010*"photography" + 0.010*"uniform" + 0.010*"flash" + 0.009*"tree" + 0.009*"font" + 0.009*"water" + 0.008*"f

0.011*"smile" + 0.010*"gear" + 0.009*"muscle" + 0.009*"human" + 0.009*"t" + 0.008*"body" + 0.008*"arm" + 0.008*"street" + 0.007*"short" + 0.007*"mountain"
0.009*"font" + 0.009*"shorts" + 0.008*"human" + 0.008*"street" + 0.008*"outdoor" + 0.007*"wood" + 0.007*"event" + 0.007*"gear" + 0.007*"blue" + 0.007*"shirt"
0.011*"street" + 0.010*"outdoor" + 0.008*"smile" + 0.007*"gear" + 0.007*"art" + 0.007*"grass" + 0.007*"font" + 0.007*"human" + 0.007*"light" + 0.007*"body"
0.011*"street" + 0.010*"art" + 0.010*"body" + 0.009*"outdoor" + 0.009*"grass" + 0.009*"design" + 0.009*"human" + 0.009*"light" + 0.008*"entertainment" + 0.008*"landscape"
0.011*"street" + 0.009*"outdoor" + 0.009*"arm" + 0.009*"grass" + 0.009*"body" + 0.008*"knee" + 0.008*"art" + 0.008*"shorts" + 0.007*"hand" + 0.007*"short"
0.010*"light" + 0.009*"font" + 0.009*"shorts" + 0.008*"shirt" + 0.008*"outdoor" + 0.008*"wood" + 0.008*"landscape" + 0.008*"grass" + 0.008*"human" + 0.008*"art"
0.012*"knee" + 0.010*"short" + 0.010*"grass"

0.010*"fashion" + 0.009*"street" + 0.008*"gear" + 0.008*"recreation" + 0.008*"tree" + 0.008*"thigh" + 0.007*"sleeve" + 0.007*"shorts" + 0.007*"smile" + 0.007*"human"
0.009*"recreation" + 0.009*"and" + 0.009*"human" + 0.009*"thigh" + 0.008*"water" + 0.008*"tree" + 0.008*"shorts" + 0.007*"leisure" + 0.007*"grass" + 0.007*"building"
0.011*"thigh" + 0.009*"water" + 0.009*"fashion" + 0.009*"leisure" + 0.009*"recreation" + 0.009*"-" + 0.008*"outdoor" + 0.008*"art" + 0.008*"and" + 0.008*"knee"
0.011*"sleeve" + 0.008*"tree" + 0.008*"thigh" + 0.008*"knee" + 0.008*"shorts" + 0.007*"smile" + 0.007*"fashion" + 0.007*"font" + 0.007*"water" + 0.007*"light"
0.010*"fashion" + 0.010*"leisure" + 0.009*"sleeve" + 0.008*"thigh" + 0.008*"tree" + 0.008*"blue" + 0.008*"shorts" + 0.007*"short" + 0.007*"-" + 0.007*"street"
0.009*"tree" + 0.008*"leisure" + 0.008*"water" + 0.007*"human" + 0.007*"thigh" + 0.007*"elbow" + 0.007*"-" + 0.007*"and" + 0.007*"font" + 0.006*"snow"
0.009*"font" + 0.008*"human" + 0.008*"l

## Image Labels + Caption

In [19]:
data = pd.read_csv("./Nike/nike_img_cap_tf.csv", index_col=0)
print(data.columns)
column_name = 'img_cap_list'
data[column_name] = data[column_name].apply(lambda x : eval(x))

Index(['really', 'shawnjohnsons', 'needle', 'toddantonyphoto', 'nosajthe',
       'broken', 'fixture', 'tired', 'collect', 'brasileira',
       ...
       'universal', 'narrow', 'bowerman', 'corner', 'rise', 'tomckean',
       'fencer', 'gomofarah', 'racer', 'img_cap_list'],
      dtype='object', length=3269)


In [20]:
n_topics_l=[3, 5, 10]
alpha_val_l=[0.5, 0.7]
eta_val_l = [None, 0.5, 0.7]
top_n_l =[None, 10, 20, 30]
writer_tf = pd.ExcelWriter('nike_topic_model_imgcap_tf.xlsx')
writer_tfidf = pd.ExcelWriter('nike_topic_model_imgcap_tfidf.xlsx')

for n_topics, alpha_val, eta_val, top_n in itertools.product(n_topics_l, alpha_val_l, eta_val_l, top_n_l):
    print('# Topics', n_topics, 'Alpha', alpha_val, 'ETA val', eta_val, 'top_n', top_n)
    sheet_name = str(n_topics) + '_' + str(alpha_val) + '_' + str(eta_val) + '_' + str(top_n)
    
    print('\n------------------------LDA without TFIDF ----------------------- \n')
    lda_cap = createLDA(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val = eta_val, top_n=top_n)
    lda_topic_weight_tf = get_word2topic(lda_cap)
    lda_topic_weight_tf.to_excel(writer_tf, sheet_name=sheet_name+'_tf', index=True)
    
    print('\n------------------------LDA with TFIDF ----------------------- \n')
    lda_cap_tfidf = createLDA_TFIDF(column_name=column_name, im_cap_tf=data, n_topics=n_topics, alpha_val=alpha_val, eta_val=eta_val, top_n=top_n)
    lda_topic_weight_tfidf = get_word2topic(lda_cap_tfidf)
    lda_topic_weight_tfidf.to_excel(writer_tfidf, sheet_name=sheet_name+'_tfidf', index=True)
    
writer_tf.save() 
writer_tfidf.save() 

# Topics 3 Alpha 0.5 ETA val None top_n None

------------------------LDA without TFIDF ----------------------- 

0.011*"shoe" + 0.011*"justdoit" + 0.009*"``" + 0.008*"sky" + 0.007*"sports" + 0.007*"sport" + 0.007*"people" + 0.007*"nature" + 0.007*"nike" + 0.006*"equipment"
0.016*"sports" + 0.012*"equipment" + 0.011*"justdoit" + 0.008*"``" + 0.008*"sky" + 0.008*"football" + 0.008*"sport" + 0.008*"player" + 0.006*"jersey" + 0.006*"world"
0.017*"justdoit" + 0.013*"shoe" + 0.012*"sky" + 0.010*"sports" + 0.009*"equipment" + 0.008*"cloud" + 0.007*"sport" + 0.006*"``" + 0.006*"sportswear" + 0.006*"people"

------------------------LDA with TFIDF ----------------------- 

0.016*"justdoit" + 0.011*"sports" + 0.011*"equipment" + 0.009*"sport" + 0.008*"``" + 0.007*"player" + 0.007*"shoe" + 0.007*"sky" + 0.007*"nature" + 0.006*"people"
0.014*"shoe" + 0.013*"sky" + 0.012*"sports" + 0.012*"justdoit" + 0.008*"people" + 0.008*"nike" + 0.008*"equipment" + 0.007*"run" + 0.007*"sportswear" + 0.006*"sport

0.005*"tree" + 0.005*"body" + 0.005*"fashion" + 0.004*"landscape" + 0.004*"'s" + 0.004*"mountain" + 0.004*"nt" + 0.004*"light" + 0.004*"natural" + 0.004*"event"
0.007*"recreation" + 0.005*"sleeve" + 0.005*"and" + 0.005*"thigh" + 0.004*"fashion" + 0.004*"outdoor" + 0.004*"human" + 0.004*"leisure" + 0.004*"tree" + 0.004*"-"
0.005*"art" + 0.005*"font" + 0.004*"make" + 0.004*"shorts" + 0.004*"leisure" + 0.004*"light" + 0.004*"thigh" + 0.004*"design" + 0.004*"gear" + 0.003*"day"

------------------------LDA with TFIDF ----------------------- 

0.005*"outdoor" + 0.005*"leisure" + 0.005*"recreation" + 0.004*"tree" + 0.004*"fashion" + 0.004*"thigh" + 0.004*"-" + 0.004*"sleeve" + 0.004*"shorts" + 0.004*"and"
0.005*"grass" + 0.005*"'s" + 0.004*"fashion" + 0.004*"light" + 0.004*"font" + 0.004*"play" + 0.004*"and" + 0.004*"thigh" + 0.003*"-" + 0.003*"shorts"
0.005*"body" + 0.005*"sleeve" + 0.004*"tree" + 0.004*"art" + 0.004*"human" + 0.004*"thigh" + 0.004*"landscape" + 0.004*"muscle" + 0.004*"gear


------------------------LDA with TFIDF ----------------------- 

0.006*"footwear" + 0.006*"thigh" + 0.005*"photography" + 0.005*"leisure" + 0.005*"make" + 0.005*"flash" + 0.005*"get" + 0.004*"street" + 0.004*"leg" + 0.004*"fashion"
0.007*"uniform" + 0.006*"street" + 0.006*"footwear" + 0.006*"flash" + 0.005*"leg" + 0.005*"photography" + 0.005*"fashion" + 0.005*"sleeve" + 0.005*"shorts" + 0.004*"jersey"
0.006*"cloud" + 0.006*"recreation" + 0.005*"uniform" + 0.005*"water" + 0.005*"jersey" + 0.004*"tree" + 0.004*"body" + 0.004*"mountain" + 0.004*"human" + 0.004*"gear"
# Topics 3 Alpha 0.7 ETA val None top_n 30

------------------------LDA without TFIDF ----------------------- 

0.005*"leisure" + 0.005*"human" + 0.005*"tree" + 0.005*"shorts" + 0.005*"fashion" + 0.005*"recreation" + 0.004*"gear" + 0.004*"grass" + 0.004*"smile" + 0.004*"knee"
0.005*"thigh" + 0.004*"font" + 0.004*"fashion" + 0.004*"sleeve" + 0.004*"body" + 0.004*"shirt" + 0.004*"make" + 0.004*"elbow" + 0.004*"new" + 0.004*"li

0.006*"cloud" + 0.005*"footwear" + 0.004*"grass" + 0.004*"tree" + 0.004*"fashion" + 0.004*"landscape" + 0.004*"water" + 0.004*"leg" + 0.004*"and" + 0.004*"light"
0.005*"uniform" + 0.005*"thigh" + 0.004*"footwear" + 0.004*"tree" + 0.004*"leisure" + 0.004*"photography" + 0.004*"muscle" + 0.004*"flash" + 0.004*"recreation" + 0.003*"leg"
0.006*"street" + 0.005*"photography" + 0.005*"flash" + 0.004*"cloud" + 0.004*"new" + 0.004*"footwear" + 0.004*"leg" + 0.004*"uniform" + 0.004*"shorts" + 0.004*"art"

------------------------LDA with TFIDF ----------------------- 

0.006*"footwear" + 0.005*"photography" + 0.004*"flash" + 0.004*"uniform" + 0.004*"cloud" + 0.004*"sleeve" + 0.004*"leg" + 0.004*"water" + 0.004*"body" + 0.003*"art"
0.005*"street" + 0.005*"uniform" + 0.005*"leg" + 0.004*"shorts" + 0.004*"cloud" + 0.004*"flash" + 0.004*"footwear" + 0.004*"photography" + 0.004*"recreation" + 0.004*"leisure"
0.004*"cloud" + 0.004*"uniform" + 0.004*"human" + 0.004*"thigh" + 0.004*"-" + 0.004*"footwea


------------------------LDA with TFIDF ----------------------- 

0.005*"human" + 0.004*"body" + 0.004*"smile" + 0.004*"'s" + 0.004*"outdoor" + 0.004*"recreation" + 0.004*"grass" + 0.004*"elbow" + 0.004*"make" + 0.004*"leisure"
0.006*"light" + 0.005*"leisure" + 0.005*"make" + 0.005*"-" + 0.005*"font" + 0.005*"gear" + 0.005*"wood" + 0.005*"and" + 0.005*"thigh" + 0.004*"art"
0.005*"fashion" + 0.005*"thigh" + 0.005*"human" + 0.005*"knee" + 0.004*"and" + 0.004*"shirt" + 0.004*"short" + 0.004*"'s" + 0.004*"smile" + 0.004*"new"
0.007*"tree" + 0.007*"thigh" + 0.005*"grass" + 0.005*"leisure" + 0.005*"art" + 0.005*"knee" + 0.004*"fashion" + 0.004*"light" + 0.004*"elbow" + 0.004*"good"
0.007*"sleeve" + 0.006*"landscape" + 0.005*"outdoor" + 0.005*"shorts" + 0.005*"mountain" + 0.004*"recreation" + 0.004*"slope" + 0.004*"and" + 0.004*"play" + 0.004*"new"
# Topics 5 Alpha 0.5 ETA val 0.5 top_n None

------------------------LDA without TFIDF ----------------------- 

0.009*"shoe" + 0.007*"justdoit" +

...


------------------------LDA with TFIDF ----------------------- 

0.002*"body" + 0.002*"and" + 0.002*"art" + 0.002*"leisure" + 0.002*"black" + 0.002*"good" + 0.002*"human" + 0.002*"blue" + 0.002*"tree" + 0.002*"play"
0.003*"and" + 0.003*"shorts" + 0.002*"mountain" + 0.002*"body" + 0.002*"sleeve" + 0.002*"thigh" + 0.002*"landscape" + 0.002*"gear" + 0.002*"go" + 0.002*"white"
0.003*"tree" + 0.003*"leisure" + 0.003*"thigh" + 0.003*"grass" + 0.003*"landscape" + 0.003*"art" + 0.003*"light" + 0.003*"sleeve" + 0.002*"community" + 0.002*"human"
0.003*"shorts" + 0.003*"thigh" + 0.002*"'s" + 0.002*"new" + 0.002*"nt" + 0.002*"play" + 0.002*"recreation" + 0.002*"grass" + 0.002*"art" + 0.002*"and"
0.005*"leisure" + 0.003*"tree" + 0.003*"short" + 0.002*"thigh" + 0.002*"shirt" + 0.002*"art" + 0.002*"recreation" + 0.002*"body" + 0.002*"human" + 0.002*"event"
0.003*"-" + 0.003*"elbow" + 0.003*"fashion" + 0.003*"hand" + 0.003*"smile" + 0.003*"sleeve" + 0.002*"and" + 0.002*"shirt" + 0.002*"t" + 0.002*"gr

# Topics 10 Alpha 0.7 ETA val None top_n 20

------------------------LDA without TFIDF ----------------------- 

0.006*"street" + 0.006*"and" + 0.005*"cloud" + 0.005*"photography" + 0.005*"uniform" + 0.005*"fashion" + 0.005*"leg" + 0.005*"flash" + 0.005*"recreation" + 0.005*"light"
0.006*"cloud" + 0.005*"thigh" + 0.005*"water" + 0.005*"recreation" + 0.005*"body" + 0.005*"uniform" + 0.005*"photography" + 0.005*"leg" + 0.005*"footwear" + 0.004*"street"
0.006*"uniform" + 0.005*"leg" + 0.005*"cloud" + 0.004*"photography" + 0.004*"leisure" + 0.004*"street" + 0.004*"fashion" + 0.004*"gear" + 0.004*"outdoor" + 0.004*"human"
0.006*"uniform" + 0.005*"photography" + 0.005*"jersey" + 0.005*"sleeve" + 0.005*"leisure" + 0.005*"footwear" + 0.004*"thigh" + 0.004*"-" + 0.004*"art" + 0.004*"nt"
0.006*"flash" + 0.006*"human" + 0.005*"uniform" + 0.005*"cloud" + 0.005*"photography" + 0.005*"leg" + 0.005*"thigh" + 0.005*"sleeve" + 0.005*"recreation" + 0.005*"'s"
0.007*"footwear" + 0.006*"leg" + 0.006*"stre


------------------------LDA with TFIDF ----------------------- 

0.010*"shoe" + 0.010*"justdoit" + 0.006*"sports" + 0.006*"equipment" + 0.005*"``" + 0.005*"people" + 0.004*"sky" + 0.004*"nature" + 0.004*"sport" + 0.004*"nike"
0.007*"``" + 0.007*"sports" + 0.007*"justdoit" + 0.005*"sky" + 0.005*"equipment" + 0.004*"shoe" + 0.004*"nike" + 0.004*"people" + 0.004*"gesture" + 0.004*"sportswear"
0.007*"sports" + 0.007*"sport" + 0.006*"justdoit" + 0.006*"shoe" + 0.006*"sky" + 0.006*"equipment" + 0.005*"``" + 0.005*"player" + 0.004*"sportswear" + 0.004*"football"
0.008*"justdoit" + 0.007*"sky" + 0.005*"people" + 0.005*"shoe" + 0.005*"sports" + 0.004*"nature" + 0.004*"sport" + 0.004*"plant" + 0.004*"equipment" + 0.004*"``"
0.009*"sports" + 0.007*"justdoit" + 0.006*"shoe" + 0.006*"equipment" + 0.006*"sky" + 0.006*"``" + 0.005*"sport" + 0.005*"player" + 0.004*"nike" + 0.004*"water"
0.011*"shoe" + 0.009*"sports" + 0.007*"equipment" + 0.007*"justdoit" + 0.005*"football" + 0.005*"sport" + 0.005*"ni

# Topics 10 Alpha 0.7 ETA val 0.5 top_n 30

------------------------LDA without TFIDF ----------------------- 

0.003*"thigh" + 0.003*"sleeve" + 0.003*"human" + 0.003*"body" + 0.003*"tree" + 0.003*"like" + 0.003*"go" + 0.003*"grass" + 0.003*"font" + 0.002*"fashion"
0.003*"light" + 0.003*"and" + 0.002*"grass" + 0.002*"leisure" + 0.002*"recreation" + 0.002*"tree" + 0.002*"human" + 0.002*"community" + 0.002*"landscape" + 0.002*"game"
0.003*"tree" + 0.003*"mountain" + 0.003*"fashion" + 0.003*"make" + 0.003*"thigh" + 0.002*"time" + 0.002*"outdoor" + 0.002*"shorts" + 0.002*"nt" + 0.002*"grass"
0.003*"thigh" + 0.003*"recreation" + 0.002*"fashion" + 0.002*"-" + 0.002*"tree" + 0.002*"sleeve" + 0.002*"leisure" + 0.002*"shorts" + 0.002*"human" + 0.002*"design"
0.003*"leisure" + 0.003*"make" + 0.003*"good" + 0.003*"body" + 0.002*"shorts" + 0.002*"tree" + 0.002*"fashion" + 0.002*"thigh" + 0.002*"and" + 0.002*"sleeve"
0.003*"thigh" + 0.003*"tree" + 0.003*"gear" + 0.003*"sleeve" + 0.002*"body" + 0.00


------------------------LDA with TFIDF ----------------------- 

0.004*"nature" + 0.004*"in" + 0.003*"happy" + 0.003*"photography" + 0.003*"fashion" + 0.003*"flash" + 0.003*"footwear" + 0.003*"world" + 0.003*"sportswear" + 0.003*"body"
0.003*"uniform" + 0.003*"sportswear" + 0.003*"gesture" + 0.003*"jersey" + 0.003*"happy" + 0.002*"gear" + 0.002*"water" + 0.002*"street" + 0.002*"nature" + 0.002*"plant"
0.004*"nature" + 0.003*"happy" + 0.003*"gesture" + 0.002*"in" + 0.002*"sportswear" + 0.002*"uniform" + 0.002*"leg" + 0.002*"plant" + 0.002*"cloud" + 0.002*"thigh"
0.004*"happy" + 0.004*"cloud" + 0.003*"photography" + 0.003*"gesture" + 0.003*"nature" + 0.003*"in" + 0.003*"get" + 0.002*"flash" + 0.002*"human" + 0.002*"sportswear"
0.004*"sportswear" + 0.003*"leisure" + 0.003*"footwear" + 0.003*"nature" + 0.003*"fashion" + 0.003*"world" + 0.002*"run" + 0.002*"photography" + 0.002*"recreation" + 0.002*"uniform"
0.003*"gesture" + 0.003*"plant" + 0.003*"sleeve" + 0.003*"tree" + 0.003*"footwear"